In [ ]:
#| default_exp configs

# Configs
> Several dictionaries used to generate `.toml` configuration files copied under `/home/.marisco` folder and associated utilities function. These `.toml` files can be then adapted to your specific needs if required.



In [ ]:
#| export
from pathlib import Path
import re
from functools import partial

from marisco.inout import read_toml, write_toml
import pandas as pd

import fastcore.all as fc

## Configuration files

In [ ]:
#| export
CFG_FNAME = 'configs.toml'
CDL_FNAME = 'cdl.toml'
NUCLIDE_LOOKUP_FNAME = 'dbo_nuclide.xlsx'
MARISCO_CFG_DIRNAME = '.marisco'

In [ ]:
#| export
def base_path(): return Path.home() / MARISCO_CFG_DIRNAME

By default, we create a folder named `.marisco` under your home directory that will receive all configuration files as defined in `BASE_PATH`:

In [ ]:
base_path()

Path('/Users/franckalbinet/.marisco')

In [ ]:
#| export
CONFIGS = {
    'gh': {
        'owner': 'franckalbinet',
        'repo': 'marisco'
    },
    'names': {
        'nc_template': 'maris-template.nc'
    },
    'dirs': {
        'lut': str(base_path() / 'lut'), # Look-up tables
        'cache': str(base_path() / 'cache'), # Cache (e.f WoRMS species)
        'tmp': str(base_path() / 'tmp')
    },
    'paths': {
        'luts': 'nbs/files/lut'
    },
    'units': {
        'time': 'seconds since 1970-01-01 00:00:00.0'
    },
    'zotero': {
        'api_key': '7nuMByrm19RP7reiQtYJwgPk',
        'lib_id': '2432820'
    }
}

The `CONFIGS` dictionary defines general settings:

| key     | Value  | Description                                   |
|---------|-------------|--------------------------------------------|
| `dirs/lut`   | `/Users/franckalbinet/.marisco/lut`         | Location & name of the directory receiving lookup tables.          |
| `dirs/cache`   | `/Users/franckalbinet/.marisco/cache`         | Location & name of the directory receiving cache files such as WoRMS species retrieved.          |
| `dirs/tmp`    |  `/Users/franckalbinet/.marisco/tmp`         | Location & name of temporary files.       |
| `gh/owner`     |  `franckalbinet`       | GitHub account owner.                             |
| `gh/repo`      |  `marisco`       |   GitHub user used to download specific files (e.g lookup tables) during installation.                                  |
| `names/nc_template`   | `maris-template.nc` |  Name of the MARIS NetCDF4 template.                         |
| `paths_luts`   |  `nbs/files/lut`        |  GitHub repository directory name containing lookup tables.                             |
| `units_time`   |    `seconds since 1970-01-01 00:00:00.0`     | Reference date and time used for [NetCDF time encoding](https://unidata.github.io/cftime/).        |
| `zotero/api_key`  | `your-zotero-api-key`      |   Zotero API key.                     |
| `zotero/lib_id`  |  `2432820`       | Zotero library ID.                                  |


The main `CONFIGS_CDL` dictionary, used to generate a [NetCDF CDL (Common Data Language)](https://www.unidata.ucar.edu/software/netcdf/workshops/2012/nc3model/Cdl.html) `.toml` file. This file is then used to generate a template MARIS netcdf file. For further details refers to the [`configs.ipynb`](https://github.com/franckalbinet/marisco/blob/main/nbs/api/configs.ipynb) file.

Below, the vars/defaults section printed:

In [ ]:
#| export
def cfg(): return read_toml(base_path() / CFG_FNAME)

In [ ]:
#| export
def nuc_lut_path(): return Path(cfg()['dirs']['lut']) / NUCLIDE_LOOKUP_FNAME

In [ ]:
#| export
def lut_path(): return Path(cfg()['dirs']['lut'])

In [ ]:
#| export
def cache_path(): return Path(cfg()['dirs']['cache'])

In [ ]:
#| export
CONFIGS_CDL = { 
    'placeholder': '_to_be_filled_in_',
    'grps': {
        'sea': {
            'name': 'seawater'
        },
        'bio': {
            'name': 'biota'
        },
        'sed': {
            'name': 'sediment'
        },
        'sus': {
            'name': 'suspended-matter'
        }
    },
    'global_attrs': {
        # Do not update keys. Only values if required
        'id': '', # zotero?
        'title': '',
        'summary': '',
        'keywords': '',
        'keywords_vocabulary': 'GCMD Science Keywords',
        'keywords_vocabulary_url': 'https://gcmd.earthdata.nasa.gov/static/kms/',
        'record': '',
        'featureType': '',
        'cdm_data_type': '',

        # Conventions
        'Conventions': 'CF-1.10 ACDD-1.3',

        # Publisher [ACDD1.3]
        'publisher_name': 'Paul MCGINNITY, Iolanda OSVATH, Florence DESCROIX-COMANDUCCI',
        'publisher_email': 'p.mc-ginnity@iaea.org, i.osvath@iaea.org, F.Descroix-Comanducci@iaea.org', 
        'publisher_url': 'https://maris.iaea.org',
        'publisher_institution': 'International Atomic Energy Agency - IAEA', 

        # Creator info [ACDD1.3]
        'creator_name': '',
        'institution': '',
        'metadata_link': '',
        'creator_email': '',
        'creator_url': '',
        'references': '',
        'license': ' '.join(['Without prejudice to the applicable Terms and Conditions', 
                             '(https://nucleus.iaea.org/Pages/Others/Disclaimer.aspx),',
                             'I hereby agree that any use of the data will contain appropriate',
                             'acknowledgement of the data source(s) and the IAEA Marine',
                             'Radioactivity Information System (MARIS).']),
        'comment': '',
        # Dataset info & coordinates [ACDD1.3]
        #'project': '', # Network long name
        #'platform': '', # Should be a long / full name
        'geospatial_lat_min': '', 
        'geospatial_lon_min': '',
        'geospatial_lat_max': '',
        'geospatial_lon_max': '',
        'geospatial_vertical_min': '',
        'geospatial_vertical_max': '',
        'geospatial_bounds': '', # wkt representation
        'geospatial_bounds_crs': 'EPSG:4326',

        # Time information
        'time_coverage_start': '',
        'time_coverage_end': '',
        #'time_coverage_resolution': '',
        'local_time_zone': '',
        'date_created': '',
        'date_modified': '',
        #
        # -- Additional metadata (custom to MARIS)
        #
        'publisher_postprocess_logs': ''
        },
    'dim': {
        'name': 'sample',
        'attrs': {
            'long_name': 'Sample ID of measurement'
        },
        'dtype': 'u8'
    },
    'vars': {    
        'defaults': {
            'lon': {
                'name': 'lon',
                'attrs': {
                    'long_name': 'Measurement longitude',
                    'standard_name': 'longitude',
                    'units': 'degrees_north',
                    'axis': 'Y',
                    '_CoordinateAxisType': 'Lon'
                },
                'dtype': 'f4'
            },
            'lat': {
                'name': 'lat',
                'attrs': {
                    'long_name': 'Measurement latitude',
                    'standard_name': 'latitude',
                    'units': 'degrees_east',
                    'axis': 'X',
                    '_CoordinateAxisType': 'Lat'
                },
                'dtype': 'f4'
            },
            'depth': {
                'name': 'depth',
                'attrs': {
                    'long_name': 'Depth below seal level',
                    'standard_name': 'depth_below_sea_floor',
                    'units': 'm',
                    'axis': 'Z'
                },
                'dtype': 'f4'
            },
            'time': {
                'name': 'time',
                'attrs': {
                    'long_name': 'Time of measurement',
                    'standard_name': 'time',
                    'units': 'seconds since 1970-01-01 00:00:00.0',
                    'time_origin': '1970-01-01 00:00:00',
                    'time_zone': 'UTC',
                    'abbreviation': 'Date/Time',
                    'axis': 'T',
                    'calendar': 'gregorian'
                },
                'dtype': 'u8',
            },
            'area': {
                'name': 'area',
                'attrs': {
                    'long_name': 'Marine area/region id',
                    'standard_name': 'area_id'
                },
                'dtype': 'area_t'
            },
        },
        'bio': {
            'bio_group': {
                'name': 'bio_group',
                'attrs': {
                    'long_name': 'Biota group',
                    'standard_name': 'biota_group_tbd'
                },
                'dtype': 'bio_group_t'
            },
            'species': {
                'name': 'species',
                'attrs': {  
                    'long_name': 'Species',
                    'standard_name': 'species'
                },
                'dtype': 'species_t'
            },
            'body_part': {
                'name': 'body_part',
                'attrs': {
                    'long_name': 'Body part',
                    'standard_name': 'body_part_tbd'
                },
                'dtype': 'body_part_t' 
            }
        },
        'sed': {
            'sed_type': {
                'name': 'sed_type',
                'attrs': {
                    'long_name': 'Sediment type',
                    'standard_name': 'sediment_type_tbd'
                },
                'dtype': 'sed_type_t'
            }
        },
        'suffixes':  {
            'uncertainty': {
                'name': '_unc',
                'attrs': {
                    'long_name': ' uncertainty',
                    'standard_name': '_uncertainty'
                },
                'dtype': 'f4'
            },
            'detection_limit': {
                'name': '_dl',
                'attrs': {
                    'long_name': ' detection limit',
                    'standard_name': '_detection_limit'
                },
                'dtype': 'dl_t'
            },
            'volume': {
                'name': '_vol',
                'attrs': {
                    'long_name': ' volume',
                    'standard_name': '_volume'
                },
                'dtype': 'f4'
            },
            'salinity': {
                'name': '_sal',
                'attrs': {
                    'long_name': ' salinity',
                    'standard_name': '_sal'
                },
                'dtype': 'f4'
            },
            'temperature': {
                'name': '_temp',
                'attrs': {
                    'long_name': ' temperature',
                    'standard_name': '_temp'
                },
                'dtype': 'f4'
            },
            'filtered': {
                'name': '_filt',
                'attrs': {
                    'long_name': ' filtered',
                    'standard_name': '_filtered'
                },
                'dtype': 'filt_t'
            },
            'counting_method': {
                'name': '_counmet',
                'attrs': {
                    'long_name': ' counting method',
                    'standard_name': '_counting_method'
                },
                'dtype': 'counmet_t'
            },
            'sampling_method': {
                'name': '_sampmet',
                'attrs': {
                    'long_name': ' sampling method',
                    'standard_name': '_sampling_method'
                },
                'dtype': 'sampmet_t'
            },
            'preparation_method': {
                'name': '_prepmet',
                'attrs': {
                    'long_name': ' preparation method',
                    'standard_name': '_preparation_method'
                },
                'dtype': 'prepmet_t'
            },
            'unit': {
                'name': '_unit',
                'attrs': {
                    'long_name': ' unit',
                    'standard_name': '_unit'
                },
                'dtype': 'unit_t'
            }
        }
    },
    'enums': [
        {
            'name': 'area_t', 
            'fname': 'dbo_area.xlsx', 
            'key': 'displayName', 
            'value':'areaId'
        },
        {
            'name': 'bio_group_t', 
            'fname': 'dbo_biogroup.xlsx', 
            'key': 'biogroup', 
            'value':'biogroup_id'
        },
        {
            'name': 'body_part_t', 
            'fname': 'dbo_bodypar.xlsx', 
            'key': 'bodypar', 
            'value':'bodypar_id'
        },
        {
            'name': 'species_t', 
            'fname': 'dbo_species_cleaned.xlsx', 
            'key': 'species', 
            'value':'species_id'
        },
        {
            'name': 'sed_type_t', 
            'fname': 'dbo_sedtype.xlsx', 
            'key': 'sedtype', 
            'value':'sedtype_id'
        },
        {
            'name': 'unit_t', 
            'fname': 'dbo_unit.xlsx', 
            'key': 'unit_sanitized', 
            'value':'unit_id'
        },
        {
            'name': 'dl_t', 
            'fname': 'dbo_detectlimit.xlsx', 
            'key': 'name_sanitized', 
            'value':'id'
        },
        {
            'name': 'filt_t', 
            'fname': 'dbo_filtered.xlsx', 
            'key': 'name',
            'value':'id'
        },
        {
            'name': 'counmet_t', 
            'fname': 'dbo_counmet.xlsx', 
            'key': 'counmet',
            'value':'counmet_id'
        },
        {
            'name': 'sampmet_t', 
            'fname': 'dbo_sampmet.xlsx', 
            'key': 'sampmet',
            'value':'sampmet_id'
        },
        {
            'name': 'prepmet_t', 
            'fname': 'dbo_prepmet.xlsx', 
            'key': 'prepmet',
            'value':'prepmet_id'
        }
        ]
}

In [ ]:
fc.AttrDict(CONFIGS_CDL['vars']['defaults'])

```json
{ 'area': { 'attrs': { 'long_name': 'Marine area/region id',
                       'standard_name': 'area_id'},
            'dtype': 'area_t',
            'name': 'area'},
  'depth': { 'attrs': { 'axis': 'Z',
                        'long_name': 'Depth below seal level',
                        'standard_name': 'depth_below_sea_floor',
                        'units': 'm'},
             'dtype': 'f4',
             'name': 'depth'},
  'lat': { 'attrs': { '_CoordinateAxisType': 'Lat',
                      'axis': 'X',
                      'long_name': 'Measurement latitude',
                      'standard_name': 'latitude',
                      'units': 'degrees_east'},
           'dtype': 'f4',
           'name': 'lat'},
  'lon': { 'attrs': { '_CoordinateAxisType': 'Lon',
                      'axis': 'Y',
                      'long_name': 'Measurement longitude',
                      'standard_name': 'longitude',
                      'units': 'degrees_north'},
           'dtype': 'f4',
           'name': 'lon'},
  'time': { 'attrs': { 'abbreviation': 'Date/Time',
                       'axis': 'T',
                       'calendar': 'gregorian',
                       'long_name': 'Time of measurement',
                       'standard_name': 'time',
                       'time_origin': '1970-01-01 00:00:00',
                       'time_zone': 'UTC',
                       'units': 'seconds since 1970-01-01 00:00:00.0'},
            'dtype': 'u8',
            'name': 'time'}}
```

In [ ]:
#| hide
write_toml(Path('./files') / CDL_FNAME, CONFIGS_CDL)

Creating files/cdl.toml


In [ ]:
#| export
def cdl_cfg(): return read_toml(base_path() / CDL_FNAME)

In [ ]:
#| export
def species_lut_path():
    src_dir = lut_path()
    fname = [enum for enum in cdl_cfg()['enums'] if enum['name'] == 'species_t'][0]['fname']
    return src_dir / fname

In [ ]:
#| export
def bodyparts_lut_path():
    src_dir = lut_path()
    fname = [enum for enum in cdl_cfg()['enums'] if enum['name'] == 'body_part_t'][0]['fname']
    return src_dir / fname

In [ ]:
#| export
def sediments_lut_path():
    src_dir = lut_path()
    fname = [enum for enum in cdl_cfg()['enums'] if enum['name'] == 'sed_type_t'][0]['fname']
    return src_dir / fname

## Utilities function

In [ ]:
#| export
NETCDF_TO_PYTHON_TYPE = {
    'u8': int,
    'f4': float
    }

In [ ]:
#| export
def name2grp(
    name:str, # Name of the group
    cdl:dict, # CDL configuration
    ):
    # Reverse `cdl.toml` config group dict so that group config key can be retrieve based on its name
    return {v['name']:k  for k, v in cdl['grps'].items()}[name]

Example:

In [ ]:
name2grp('seawater', cdl=CONFIGS_CDL)

'sea'

In [ ]:
#| export
def nc_tpl_name():
    p = base_path()
    return read_toml(p / 'configs.toml')['names']['nc_template']

In [ ]:
#| export
def nc_tpl_path():
    "Return the name of the MARIS NetCDF template as defined in `configs.toml`"
    p = base_path()
    return p / read_toml(p / 'configs.toml')['names']['nc_template']

## Enumeration types

Enumeration types are used to avoid using strings as NetCDF4 variable values. Instead, enumeration types (lookup tables) such as `{'Crustaceans': 2, 'Echinoderms': 3, ...}` are prepended to the NetCDF file template and associated ids (integers) are used as values.

In [ ]:
#| export
def sanitize(s:str # String to sanitize
             ) -> str:
    """
    Sanitize dictionary key to comply with NetCDF enumeration type: 
    
        - remove `(`, `)`, `.`, `/`, `-`  
        - strip the string
    """
    s = re.sub(r'[().]', '', s)
    return re.sub(r'[/-]', ' ', s).strip() 

For example:

In [ ]:
fc.test_eq(sanitize('key (sanitized)'), 'key sanitized')
fc.test_eq(sanitize('key san.itized'), 'key sanitized')
fc.test_eq(sanitize('key-sanitized'), 'key sanitized')
fc.test_eq(sanitize('key/sanitized'), 'key sanitized')

NetCDF4 enumeration type seems to not accept keys containing non alphanumeric characters like parentheses, dots, slash, ... As a result, MARIS lookup table needs to be sanitized.

In [ ]:
#| export
def get_lut(src_dir:str, # Directory containing lookup tables
            fname:str, # Excel file lookup table name
            key:str, # Excel file column name to be used as dict keys 
            value:str, # Excel file column name to be used as dict values 
            do_sanitize:bool=True # Sanitization required?
            ) -> dict: # MARIS lookup table
    "Convert MARIS db lookup table excel file to dictionary `{'name': id, ...}`."
    fname = Path(src_dir) / fname
    df = pd.read_excel(fname, usecols=[key, value]).dropna(subset=value)
    df[value] = df[value].astype('int')
    df = df.set_index(key)
    lut = df[value].to_dict()
    if do_sanitize: lut = {sanitize(k): v for k, v in lut.items()}
    return lut

For example:

In [ ]:
lut_src_dir = './files/lut'
get_lut(lut_src_dir, 'dbo_biogroup.xlsx', key='biogroup', value='biogroup_id')

{'Not applicable': -1,
 'Not available': 0,
 'Birds': 1,
 'Crustaceans': 2,
 'Echinoderms': 3,
 'Fish': 4,
 'Mammals': 5,
 'Molluscs': 6,
 'Others': 7,
 'Plankton': 8,
 'Polychaete worms': 9,
 'Reptile': 10,
 'Seaweeds and plants': 11,
 'Cephalopods': 12,
 'Gastropods': 13,
 'Bivalves': 14}

In [ ]:
#| export
class Enums():
    "Return dictionaries of MARIS NetCDF's enumeration types"
    def __init__(self, 
               lut_src_dir:str,
               cdl_enums:dict
               ):
        fc.store_attr()
        self.types = self.lookup()
        
    def filter(self, name, values):
        return {name: id for name, id in self.types[name].items() if id in values}
    
    def lookup(self):
        types = {}
        for enum in self.cdl_enums:
            name, fname, key, value = enum.values()
            lut = get_lut(self.lut_src_dir, fname, key=key, value=value)
            types[name] = lut
        return types

In [ ]:
lut_src_dir_test = './files/lut'
cdl_enums_test = read_toml('./files/cdl.toml')['enums']

enums = Enums(lut_src_dir=lut_src_dir_test, 
              cdl_enums=cdl_enums_test)

In [ ]:
enums.types['dl_t']

{'Not applicable': -1,
 'Not available': 0,
 'Detected value': 1,
 'Detection limit': 2,
 'Not detected': 3,
 'Derived': 4}

In [ ]:
enums.types['unit_t']

{'Not applicable': -1,
 'NOT AVAILABLE': 0,
 'Bq per m3': 1,
 'Bq per m2': 2,
 'Bq per kg': 3,
 'Bq per kgd': 4,
 'Bq per kgw': 5,
 'kg per kg': 6,
 'TU': 7,
 'DELTA per mill': 8,
 'atom per kg': 9,
 'atom per kgd': 10,
 'atom per kgw': 11,
 'atom per l': 12,
 'Bq per kgC': 13}

In [ ]:
# {f'{var}_t': enums.subset('species_t', tfm.unique(var) for var in ['species']}

In [ ]:
# subset = {}
# for var, values in [('species', [21,   50,   59,   84]),
#                     ('area', [2, 3])]:
#     # values = tfm.unique(var)
#     type_name = f'{var}_t'
#     subset[type_name] = enums.filter(type_name, [21,   50,   59,   84])

In [ ]:
# subset

In [ ]:
#| export
def get_enum_dicts(
    lut_src_dir:str,
    cdl_enums:dict,
    **kwargs
    ):
    "Return a dict of NetCDF enumeration types"
    enum_types = {}
    for enum in cdl_enums:
        name, fname, key, value = enum.values()
        lut = get_lut(lut_src_dir, fname, key=key, value=value, **kwargs)
        enum_types[name] = lut
        
    return enum_types

For example:

In [ ]:
#|eval: false
lut_src_dir_test = './files/lut'
cdl_enums_test = read_toml('./files/cdl.toml')['enums']

enums = get_enum_dicts(lut_src_dir=lut_src_dir_test, 
                       cdl_enums=cdl_enums_test); enums

{'area_t': {'Not applicable': -1,
  'Not available': 0,
  'North Atlantic Ocean': 1912,
  'Baltic Sea': 2,
  'Mediterranean Region': 3,
  'South Atlantic Ocean': 1914,
  'Indian Ocean': 1904,
  'South China & Eastern Archipelagic Seas': 6,
  'North Pacific Ocean': 1908,
  'South Pacific Ocean': 1910,
  'Arctic Ocean': 1906,
  'Southern Ocean': 1907,
  'Arctic Sea': 18,
  'Atlantic, Northwest': 21,
  'Atlantic, Northeast': 27,
  'Atlantic, Western Central': 31,
  'Atlantic, Eastern Central': 34,
  'Mediterranean and Black Sea': 37,
  'Atlantic, Southwest': 41,
  'Atlantic, Southeast': 47,
  'Atlantic, Antarctic': 48,
  'Indian Ocean, Western': 51,
  'Indian Ocean, Eastern': 57,
  'Indian Ocean, Antarctic': 58,
  'Pacific, Northwest': 61,
  'Pacific, Northeast': 67,
  'Pacific, Western Central': 71,
  'Pacific, Eastern Central': 77,
  'Pacific, Southwest': 81,
  'Pacific, Southeast': 87,
  'Pacific, Antarctic': 88,
  'North Sea': 2350,
  'Celtic Sea': 2351,
  'Norwegian Sea': 2353,
  'Gr

In [ ]:
enums['species_t']

{'Not applicable': -1,
 'NOT AVAILABLE': 0,
 'Aristeus antennatus': 1,
 'Apostichopus': 2,
 'Saccharina japonica var religiosa': 3,
 'Siganus fuscescens': 4,
 'Alpheus dentipes': 5,
 'Hexagrammos agrammus': 6,
 'Ditrema temminckii': 7,
 'Parapristipoma trilineatum': 8,
 'Scombrops boops': 9,
 'Pseudopleuronectes schrenki': 10,
 'Desmarestia ligulata': 11,
 'Saccharina japonica': 12,
 'Neodilsea yendoana': 13,
 'Costaria costata': 14,
 'Sargassum yezoense': 15,
 'Acanthephyra pelagica': 16,
 'Sargassum ringgoldianum': 17,
 'Acanthephyra quadrispinosa': 18,
 'Sargassum thunbergii': 19,
 'Sargassum patens': 20,
 'Asterias rubens': 21,
 'Sargassum miyabei': 22,
 'Homarus gammarus': 23,
 'Acanthephyra stylorostratis': 24,
 'Acanthocybium solandri': 25,
 'Acanthopagrus bifasciatus': 26,
 'Acanthophora muscoides': 27,
 'Acanthophora spicifera': 28,
 'Acanthurus triostegus': 29,
 'Actinopterygii': 30,
 'Adamussium colbecki': 31,
 'Ahnfeltiopsis densa': 32,
 'Alepes melanoptera': 33,
 'Ampharet